In [2]:
from importlib import reload
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import os

# print(os.getenv("SEVERA_CLIENT_SCOPE"))

import pandas as pd
import altair as alt

import src.severa.base_client
import src.severa.fetch
import src.database
from src.daterange import DateRange
import datetime

reload(src.severa.base_client)
reload(src.severa.fetch)
reload(src.database)

span = DateRange(540)
alt.data_transformers.disable_max_rows()

# async with src.severa.fetch.Fetcher() as f:
#    df = await f.get_resource_allocations(span)

DataTransformerRegistry.enable('default')

In [3]:
async with src.severa.fetch.Fetcher() as f:
    df2 = await f.get_allocations_with_maxes(span)

df2.head()

2023-06-02 07:44:39.076 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET users [0]: Response (9) in 0.27s.
2023-06-02 07:44:39.334 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (0) in 0.23s.
2023-06-02 07:44:39.452 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (20) in 0.36s.
2023-06-02 07:44:39.488 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (1) in 0.39s.
2023-06-02 07:44:39.496 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (0) in 0.41s.
2023-06-02 07:44:39.661 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (1) in 0.16s.
2023-06-02 07:44:39.692 | SUCCESS  | src.severa.base_client:get_with_retries:126 - HTTP/1.1 GET activities [0]: Response (0) in 0.59s.
2023-06-02 07:44:39.711 | SUCCESS  | src.severa.base_client

,businessunit-user,value,user,project,phase,date,id,forecast-date,type
96,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-03 00:00:00+00:00,external
97,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-04 00:00:00+00:00,external
98,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-05 00:00:00+00:00,external
99,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-06 00:00:00+00:00,external
100,TIE,0.642713,eb8d73da-8c1c-7996-b085-332e777dff1a,24e8dd29-d61b-c481-5013-0ff872b54c5d,979f7378-a78f-5d9a-2498-ef3550b71aa4,2023-06-02 00:00:00+00:00,allocation,2023-06-07 00:00:00+00:00,external


In [212]:
data = src.database.Base("kpi-dev", "allocations").find()

2023-06-02 13:29:45.860 | INFO     | src.database:find:32 - Query '{}' resulted in 87536 results in 10.47s.


In [211]:
g = data.groupby(["date", "forecast-date", "type"])["value"].sum().reset_index()
g["span"] = (g["forecast-date"] - g["date"]).map(lambda x: x.days)

gx = g[g["span"] > 0]
gx = gx.pivot_table(
    columns="type", values="value", index=["date", "forecast-date", "span"]
).reset_index()
gx = gx.fillna({"external": 0, "internal": 0}).dropna(subset=["max"])
gx["total"] = gx["internal"] + gx["external"]
gx["unallocated"] = gx["max"] - gx["total"]
gx["billing-rate"] = gx["external"] / gx["total"]
gx["allocation-rate"] = gx["total"] / gx["max"]
gy = gx.melt(id_vars=["date", "span"], value_vars=["external", "internal", "max", "unallocated"])
g3 = gy.merge(gx.drop(["external", "internal"], axis=1), on=["date", "span"])
g3["type"] = g3["type"].astype("category").cat.rename_categories(
    {'external':'Projektityö', 'internal':'Sisäinen työ', 'max':'Maksimi', 'unallocated':'Allokoimaton'})

# print(g3[g3["type"].isin(["internal", "external"])])


slider = alt.binding_range(min=7, max=360, step=1, name="Ennusteen pituus (vrk) xx:  ")
op_span = alt.param(value=30, bind=slider)

base = alt.Chart(g3).transform_filter((alt.datum.span >= 0) & (alt.datum.span <= op_span))

rule = alt.Chart().mark_rule(color="white", strokeDash=[4, 4]).encode(y=alt.datum(0.7))
rule_text = (
    base
    .transform_aggregate(mindate="min(date)")
    .mark_text(baseline="top", dy=5, dx=5, align="left", color="white")
    .encode(y=alt.datum(0.7), x="monthdate(mindate):T", text=alt.datum("Laskutusastetavoite 70%"))
)

chart_base = base.transform_calculate(
    order="{'Projekityö':0, 'Sisäinen työ':1, 'Maksimi':3, 'Allokoimaton':2}[datum.variable]"
).encode(
    x=alt.X("monthdate(date):T").axis(title="Päiväys"),
    y=alt.Y("sum(value):Q").axis(title="Allokoitu tuntimäärä (h)"),
    color=alt.Color("type:N", title="Tuntilaji").scale(scheme='category20c'),
    order="order:O",
    tooltip=[
        alt.Tooltip("sum(value):Q", title="Tuntimäärä (h)", format=".1f"),
        alt.Tooltip("type:N", title="Tyyppi"),
        alt.Tooltip("sum(total):Q", title="Allokoinnit yhteensä (h)", format=".1f"),
        alt.Tooltip("sum(unallocated):Q", title="Allokoimatonta (h)", format=".1f"),
        alt.Tooltip("sum(max):Q", title="Allokoinnit korkeintaan (h)", format=".1f"),
        alt.Tooltip("mean(billing-rate):Q", title="Laskutusaste", format=".1%"),
        alt.Tooltip("mean(allocation-rate):Q", title="Resursointiaste", format=".1%")
    ],
).properties(width="container")



allocations_per_type = chart_base.mark_area(
            opacity=1.0,
            point=alt.OverlayMarkDef(filled=False, fill="white", size=100)
        ).transform_filter(
            (alt.datum.type == "Sisäinen työ") | (alt.datum.type == "Projektityö")
        )

maximum_allocations = chart_base.mark_line(
            point=alt.OverlayMarkDef(filled=True, size=100), strokeDash=[4, 4]
        ).transform_filter(alt.datum.type == "Maksimi")

normalized_allocations_per_type = chart_base.mark_area().encode(
            y=alt.Y("sum(value):Q", title="Allokointi (%)").stack("normalize")
        ).transform_filter(
            alt.FieldOneOfPredicate("type", ["Projektityö", "Sisäinen työ"])
        )

(
    (
        allocations_per_type        + maximum_allocations
    ).properties(
        height=260,
    )
    & (
        normalized_allocations_per_type
        + rule
        + rule_text
    ).properties(height=80)
).add_params(op_span).interactive() # .add_params(op_span)

alt.VConcatChart(...)

In [215]:
from src.visualization import ChartGroup


await ChartGroup(30).allocated_hours(data)

TypeError: ChartGroup.treb() takes 0 positional arguments but 1 was given

alt.VConcatChart(...)

In [80]:
xxx = (
    alt.Chart(g)
    .transform_filter((alt.datum.span <= op_span) & (alt.datum.span >= 0))
    .transform_joinaggregate(total="sum(value)", groupby=["date"])
    .transform_calculate(freq=alt.datum.value / alt.datum.total)
    .add_params(op_span)
    .transform_filter(alt.datum.type != "max")
    .mark_line()
    .encode(
        x="monthdate(date):T",
        y="freq:Q",
        color="type:N",
        tooltip=[
            alt.Tooltip("type:N", title="Tyyppi"),
            alt.Tooltip("sum(value):Q", title="H"),
            alt.Tooltip("mean(freq):Q", title="%"),
            alt.Tooltip("total:Q", title="Total"),
        ],
    )
)


from altair_transform import extract_data

data = extract_data(
    alt.Chart(g[g["type"] != "max"])
    .transform_joinaggregate(total="sum(value)", groupby=["date"])
    .encode(x="date:T", y="freq:Q")
)
data.sample(10)

,date,forecast-date,type,value,span,total
927,2023-05-30,2023-08-06,external,14.631551,68,6464.550000
777,2023-05-30,2023-06-17,external,20.264645,18,6464.550000
4868,2023-06-01,2024-09-21,external,0.602721,478,4231.506253
300,2023-05-30,2022-08-28,external,0.168813,-275,6464.550000
5728,2023-06-02,2023-12-01,external,9.984751,182,4180.220903
4636,2023-06-01,2024-05-28,external,2.514579,362,4231.506253
3544,2023-05-31,2024-12-27,external,0.474217,576,4258.717671
4740,2023-06-01,2024-07-19,external,1.428720,414,4231.506253
5693,2023-06-02,2023-11-19,internal,4.035099,170,4180.220903
4790,2023-06-01,2024-08-13,external,0.671174,439,4231.506253


In [126]:
g2 = g[g["span"] > 0].pivot_table(
    columns="type", values="value", index=["date", "span"]
)
g2 = g2.reset_index().dropna(subset="max").fillna(0)
g2["total"] = g2["internal"] + g2["external"]
g2["billing-rate"] = g2["external"] / g2["total"]
g2["allocation-rate"] = g2["total"] / g2["max"]
g2.melt(id_vars=["date", "span", "total", "allocation-rate", "billing-rate"])

,date,span,total,allocation-rate,billing-rate,type,value
0,2023-05-30,1,27.447197,0.508281,0.730185,external,20.041522
1,2023-05-30,2,27.211418,0.503915,0.770705,external,20.971962
2,2023-05-30,3,27.211418,0.503915,0.770705,external,20.971962
3,2023-05-30,4,25.610484,inf,0.791264,external,20.264645
4,2023-05-30,5,25.610484,inf,0.791264,external,20.264645
...,...,...,...,...,...,...,...
4858,2023-06-01,536,0.474217,0.007711,1.000000,max,61.500000
4859,2023-06-01,537,0.474217,0.007711,1.000000,max,61.500000
4860,2023-06-01,538,0.474217,0.007711,1.000000,max,61.500000
4861,2023-06-01,539,0.474217,0.007711,1.000000,max,61.500000


In [36]:
g = grouped.pivot(columns="type", values="value", index="date").reset_index()
g

type,date,external,internal,max
0,2023-05-31 00:00:00+00:00,626.257431,169.568058,1330.5


In [43]:
g["total"] = g["external"] + g["internal"]
g["billing-rate"] = g["external"] / g["total"]
g["allocation-rate"] = g["total"] / g["max"]
g

type,date,external,internal,max,total,billing-rate,allocation-rate
0,2023-05-31 00:00:00+00:00,626.257431,169.568058,1330.5,795.825489,0.786928,0.59814


In [42]:
g.melt(id_vars=["date"])

,date,type,value
0,2023-05-31 00:00:00+00:00,external,626.257431
1,2023-05-31 00:00:00+00:00,internal,169.568058
2,2023-05-31 00:00:00+00:00,max,1330.5
3,2023-05-31 00:00:00+00:00,total,795.825489
4,2023-05-31 00:00:00+00:00,billing-rate,0.786928
5,2023-05-31 00:00:00+00:00,allocation-rate,0.59814


In [30]:
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import src.database

reload(src.database)

b = src.database.Base("kpi-dev", "allocations")

In [31]:
x = b.find()

2023-05-31 09:01:45.483 | INFO     | src.database:find:32 - Query '{}' resulted in 23438 results in 12.39s.


In [32]:
x.loc[:, ["forecast-date", "value"]]

,forecast-date,value
0,2023-02-27,0.642713
1,2023-02-28,0.642713
2,2023-03-01,0.642713
3,2023-03-02,0.642713
4,2023-03-03,0.642713
...,...,...
23433,2023-06-26,0.333333
23434,2023-06-27,0.333333
23435,2023-06-28,0.333333
23436,2023-06-29,0.333333


In [227]:
# Quick mock data
a = pd.Series(1, index=pd.date_range("2023-01-01", "2023-01-05", freq="D"))
b = pd.Series(1, index=pd.date_range("2023-01-03", "2023-01-08", freq="D"))
c = pd.Series(1, index=pd.date_range("2023-01-04", "2023-01-12", freq="D"))

df = pd.DataFrame({"user": ["A", "B", "B"], "unit": ["TIE"] * 3, "values": [a, b, c]})


# Add series with datetime indices together
def combine_series(df):
    return reduce(partial(pd.Series.add, fill_value=0), df)


# This works nicely...
# ok = combine_series(df["values"])

# ...outputs a DF, indices are OK:
# 2023-01-01    1.0
# 2023-01-02    1.0
# 2023-01-03    2.0
# 2023-01-04    3.0
# ...
# 2023-01-12    1.0
# Freq: D, dtype: float64

# This 'works' in that the values are summed correctly,
# but the indices are gone now
# not_ok = df.groupby(["unit", "user"], as_index=False).agg(combine_series)

# This is seemingly a np array
# not_ok.loc["B", "values"]
# array([1., 2., 2., 2., 2., 2., 1., 1., 1., 1.])

grouping = ["unit", "user"]
grouped = df.groupby(grouping)

pd.DataFrame(
    {"values": combine_series(v["values"])} | dict(zip(grouping, k)) for k, v in grouped
)

,values,unit,user
0,2023-01-01 1 2023-01-02 1 2023-01-03 ...,TIE,A
1,2023-01-03 1.0 2023-01-04 2.0 2023-01-05...,TIE,B


In [298]:


hours_per_day = 1.5
index=pd.date_range(
    "2023-05-01",
    "2023-07-30",
    freq="D",
)
pd.DataFrame(index=index, data={"date":"2023-05-20", "user": "ville","value":hours_per_day})[]

,date,user,value
2023-05-01,2023-05-20,ville,1.5
2023-05-02,2023-05-20,ville,1.5
2023-05-03,2023-05-20,ville,1.5
2023-05-04,2023-05-20,ville,1.5
2023-05-05,2023-05-20,ville,1.5
...,...,...,...
2023-07-26,2023-05-20,ville,1.5
2023-07-27,2023-05-20,ville,1.5
2023-07-28,2023-05-20,ville,1.5
2023-07-29,2023-05-20,ville,1.5
